In [1]:
import gc
from itertools import cycle, islice
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
tr = pd.read_csv('../input/lish-moa/train_features.csv')
te = pd.read_csv('../input/lish-moa/test_features.csv')
tr['part']='train'
te['part']='test'
df = pd.concat([tr, te], axis=0)
cs_a = ["#264653", "#2a9d8f", "#e9c46a", "#f4a261", "#e76f51"]
cs_b = ["#ff595e", "#ffca3a","#8ac926","#1982c4","#6a4c93"]
cs_c = ["#c0c", "#0cc", "#cc0", "#000"]
del tr, te
gc.collect()

0

In [3]:
i_feats = ['sig_id']
x_feats = ['cp_type', 'cp_time', 'cp_dose']
g_feats = [f'g-{i}' for i in range(770)] # 772
c_feats = [f'c-{i}' for i in range(100)]

## cp_timeごとに、各C特徴で訓練/テストの分布を比較

In [4]:
def compareplot(colname, colset, feats):
    feats_ = np.array(feats).reshape((-1,10))
    for batch_idx, feats_batch in enumerate(feats_):
        num_c = len(feats_batch)
        fig, axes = plt.subplots(num_c, 3, figsize=(5,10), sharex=True, sharey=True)
        fig.suptitle("train(magenta) vs test(yellow)")
        for j, ci in enumerate(feats_batch):
            print(f'batch {batch_idx:>3} / {len(feats_)}, plotting {j:>3} / {num_c:>3}', end='\r')
            for i, t in enumerate(colset):
                df.query(f'{colname} == {t} & part == "train"')[ci].plot(kind='kde', ax=axes[j][i], label="train", color=cs_c[0])
                df.query(f'{colname} == {t} & part == "test"')[ci].plot(kind='kde', ax=axes[j][i], label="test", color=cs_c[2])
                axes[j][i].grid(True)
            axes[j][0].set_ylabel(ci)
        for i, t in enumerate(colset):
            axes[0][i].set_title(f'time: {t}')
            axes[0][i].set_xlim(-10,10)
        plt.savefig(f"image/tr-te_feats_by_{colname}_{feats_batch[0]}.pdf")
        plt.close()
cp_times = [24,48,72]
compareplot('cp_time', cp_times, c_feats)